In [ ]:
import os
# os.chdir('../')

In [ ]:
import pandas as pd
import numpy as np
import gzip
from glob import glob
import yaml
import scipy.stats as st

## Export results for simple gene-based collapsing approach using the score-test and calculate lambda

In [ ]:
from script.python.util.snake import clean_str

import yaml

with open('./conf/config.yaml') as f:
    config = yaml.load(f, Loader=yaml.FullLoader)

if config['phenotypes'].endswith('.gz'):
    with gzip.open(config['phenotypes'],'rt') as f:
        phenotypes = f.readline().rstrip().split('\t')
else:
    with open(config['phenotypes'], 'r') as f:
        phenotypes = f.readline.rstrip().split('\t')

phenotypes = { clean_str(p): p  for p in phenotypes[1:] }

In [ ]:
def estimate_lambda(pv):

    '''
    estimate the lambda for a given array of P-values
    ------------------------------------------------------------------
    pv          numpy array containing the P-values
    ------------------------------------------------------------------
    L           lambda value
    ------------------------------------------------------------------
    '''
    
    LOD2 = np.median(st.chi2.isf(np.abs(pv[~np.isnan(pv)]), 1))
    L = (LOD2/st.chi2(1).median())
    
    return L

In [ ]:
outfiles = glob('./work/association/baseline_scoretest/all/*/results_*')

In [ ]:
pheno = [ phenotypes[s.split('/')[-2]] for s in outfiles ]

In [ ]:
filterhc = [ s.split('/')[-3] for s in outfiles ]

In [ ]:
results = [ pd.read_csv(s, sep='\t', na_values='.') for s in outfiles ]

In [ ]:
for i, p in enumerate(pheno):
    results[i]['pheno'] = p

In [ ]:
results = pd.concat(list(r for i, r in enumerate(results) if filterhc[i] == 'all')) 

In [ ]:
pd.set_option('display.max_rows',100)

In [ ]:
results = results[np.maximum(results.nCarrier_pLOF, results.nCarrier_missense) >= 5]

In [ ]:
results.sort_values('pv_mrg').head(20)

In [ ]:
results.rename(columns={'pv_pLOF':'pv_score_linb_pLOF','pv_missense':'pv_score_linb_miss','pv_mrg':'pv_score_linb_mrgLOF_miss'}, inplace=True) # renaming according to convention

In [ ]:
pv_cols = list(c for c in results.columns if c.startswith('pv_'))

In [ ]:
pv_cols

In [ ]:
lambda_val =  results.groupby(['pheno'])[pv_cols].agg(estimate_lambda)

In [ ]:
lambda_val

In [ ]:
lambda_val.to_csv('results/tables/lambdaval_baseline.tsv', sep='\t')
results.to_csv('results/tables/results_baseline.tsv.gz', sep='\t', index=False)